In [1]:
!pip --version

pip 23.2.1 from C:\ProgramData\anaconda3\Lib\site-packages\pip (python 3.11)



In [2]:
!pip install pygame

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import pygame
import sys
import random
from pygame.locals import *

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
pygame.init()

(5, 0)

In [5]:
WIDTH, HEIGHT = 300, 300
LINE_WIDTH = 15
BOARD_ROWS, BOARD_COLS = 3, 3
SQUARE_SIZE = WIDTH // BOARD_COLS
CIRCLE_RADIUS = SQUARE_SIZE // 3
CIRCLE_WIDTH = 15
CROSS_WIDTH = 25
SPACE = SQUARE_SIZE // 4
FONT_SIZE = 40
BLUR_RADIUS = 10 

BG_COLOR = (28, 170, 156)
LINE_COLOR = (23, 145, 135)
CIRCLE_COLOR = (239, 231, 200)
CROSS_COLOR = (66, 66, 66)
TEXT_COLOR = (255, 255, 255)
TRANSPARENT_COLOR = (0, 0, 0, 0)

font = pygame.font.Font(None, FONT_SIZE)

In [6]:
def draw_lines():
    pygame.draw.line(screen, LINE_COLOR, (0, SQUARE_SIZE), (WIDTH, SQUARE_SIZE), LINE_WIDTH)
    pygame.draw.line(screen, LINE_COLOR, (0, 2 * SQUARE_SIZE), (WIDTH, 2 * SQUARE_SIZE), LINE_WIDTH)
    pygame.draw.line(screen, LINE_COLOR, (SQUARE_SIZE, 0), (SQUARE_SIZE, HEIGHT), LINE_WIDTH)
    pygame.draw.line(screen, LINE_COLOR, (2 * SQUARE_SIZE, 0), (2 * SQUARE_SIZE, HEIGHT), LINE_WIDTH)

In [7]:
def initialize_board():
    board = [['' for _ in range(BOARD_COLS)] for _ in range(BOARD_ROWS)]
    return board

In [8]:
def draw_figures():
    for row in range(BOARD_ROWS):
        for col in range(BOARD_COLS):
            if board[row][col] == 'X':
                pygame.draw.line(screen, CROSS_COLOR, (col * SQUARE_SIZE + SPACE, row * SQUARE_SIZE + SQUARE_SIZE - SPACE),
                                 (col * SQUARE_SIZE + SQUARE_SIZE - SPACE, row * SQUARE_SIZE + SPACE), CROSS_WIDTH)
                pygame.draw.line(screen, CROSS_COLOR, (col * SQUARE_SIZE + SPACE, row * SQUARE_SIZE + SPACE),
                                 (col * SQUARE_SIZE + SQUARE_SIZE - SPACE, row * SQUARE_SIZE + SQUARE_SIZE - SPACE), CROSS_WIDTH)
            elif board[row][col] == 'O':
                pygame.draw.circle(screen, CIRCLE_COLOR, (col * SQUARE_SIZE + SQUARE_SIZE // 2, row * SQUARE_SIZE + SQUARE_SIZE // 2), CIRCLE_RADIUS, CIRCLE_WIDTH)

In [9]:
def mark_square(row, col, player):
    board[row][col] = player

In [10]:
def available_square(row, col):
    return board[row][col] == ''

In [11]:
def is_board_full():
    for row in range(BOARD_ROWS):
        for col in range(BOARD_COLS):
            if board[row][col] == '':
                return False
    return True

In [12]:
def check_win(player):
    for row in range(BOARD_ROWS):
        if board[row][0] == board[row][1] == board[row][2] == player:
            draw_win_line((0, row), (2, row))
            return True
    for col in range(BOARD_COLS):
        if board[0][col] == board[1][col] == board[2][col] == player:
            draw_win_line((col, 0), (col, 2))
            return True
    if board[0][0] == board[1][1] == board[2][2] == player:
        draw_win_line((0, 0), (2, 2))
        return True
    if board[0][2] == board[1][1] == board[2][0] == player:
        draw_win_line((0, 2), (2, 0))
        return True
    return False

In [13]:
def draw_win_line(start, end):
    start_x = start[0] * SQUARE_SIZE + SQUARE_SIZE // 2
    start_y = start[1] * SQUARE_SIZE + SQUARE_SIZE // 2
    end_x = end[0] * SQUARE_SIZE + SQUARE_SIZE // 2
    end_y = end[1] * SQUARE_SIZE + SQUARE_SIZE // 2
    pygame.draw.line(screen, (255, 0, 0), (start_x, start_y), (end_x, end_y), 4)

In [14]:
def draw_vertical_winner_line(col, player):
    posX = col * SQUARE_SIZE + SQUARE_SIZE // 2
    color = CROSS_COLOR if player == 'X' else CIRCLE_COLOR
    pygame.draw.line(screen, color, (posX, 15), (posX, HEIGHT - 15), 4)

In [15]:
def draw_horizontal_winner_line(row, player):
    posY = row * SQUARE_SIZE + SQUARE_SIZE // 2
    color = CROSS_COLOR if player == 'X' else CIRCLE_COLOR
    pygame.draw.line(screen, color, (15, posY), (WIDTH - 15, posY), 4)

In [16]:
def draw_diagonal_winner_line(player):
    if player == 'X':
        pygame.draw.line(screen, CROSS_COLOR, (15, 15), (WIDTH - 15, HEIGHT - 15), 4)
        pygame.draw.line(screen, CROSS_COLOR, (15, HEIGHT - 15), (WIDTH - 15, 15), 4)
    else:
        pygame.draw.line(screen, CIRCLE_COLOR, (15, 15), (WIDTH - 15, HEIGHT - 15), 4)
        pygame.draw.line(screen, CIRCLE_COLOR, (15, HEIGHT - 15), (WIDTH - 15, 15), 4)

In [17]:
def restart():
    screen.fill(BG_COLOR)
    draw_lines()
    for row in range(BOARD_ROWS):
        for col in range(BOARD_COLS):
            board[row][col] = ''
    draw_figures()

In [18]:
def computer_move():
    while True:
        row = random.randint(0, 2)
        col = random.randint(0, 2)
        if available_square(row, col):
            mark_square(row, col, 'O')
            break

In [19]:
def display_winner(winner):
    background_surface = pygame.Surface(screen.get_size())
    background_surface.blit(screen, (0, 0))
    background_surface = pygame.transform.smoothscale(background_surface, (WIDTH // BLUR_RADIUS, HEIGHT // BLUR_RADIUS))
    background_surface = pygame.transform.smoothscale(background_surface, (WIDTH, HEIGHT))
    background_surface.set_alpha(150)

    screen.blit(background_surface, (0, 0))

    text_surface = font.render(f'{winner} wins!', True, TEXT_COLOR)
    text_rect = text_surface.get_rect(center=(WIDTH // 2, HEIGHT // 2))
    screen.blit(text_surface, text_rect)

    pygame.display.update()
    pygame.time.wait(3000)

In [20]:
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Tic Tac Toe")
screen.fill(BG_COLOR)
draw_lines()

In [21]:
player = 'X'
game_over = False
board = initialize_board()

In [22]:
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        elif event.type == pygame.MOUSEBUTTONDOWN and not game_over and player == 'X':
            mouseX = event.pos[0]  # x
            mouseY = event.pos[1]  # y
            clicked_row = mouseY // SQUARE_SIZE
            clicked_col = mouseX // SQUARE_SIZE
            if available_square(clicked_row, clicked_col):
                mark_square(clicked_row, clicked_col, player)
                if check_win(player):
                    game_over = True
                elif is_board_full():
                    game_over = True
                else:
                    player = 'O'
        elif player == 'O' and not game_over:
            computer_move()
            if check_win('O'):
                game_over = True
            elif is_board_full():
                game_over = True
            else:
                player = 'X'
    
    draw_figures()
    
    if game_over:
        if check_win('X'):
            display_winner('Human player (X)')
        elif check_win('O'):
            display_winner('Computer player (O)')
        elif is_board_full():
            display_winner("It's a tie!")
        restart()
    else:
        pygame.display.update()

SystemExit: 

C:\ProgramData\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
